# Challenge: Boston marathon

There is a lot of information on runners and their performance for the Boston Marathon. Pick a year (post-2012 has more info) and do some clustering.

Specifically, use the tools at hand to determine which clustering solution, including number of clusters and algorithm used, is best for the marathon data. Once you have a solution you like, write a data story, including visualizations, where you teach the reader something about the Boston Marathon based on your clusters. Write up your report, including your process from start to finish, in a Jupyter notebook and submit it below.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('float_format', '{:.2f}'.format)
#pd.set_option('display.max_rows', None)
import pandas_profiling as pp
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
# Data Source: https://github.com/llimllib/bostonmarathon/tree/master/results/2014
df = pd.read_csv("data/boston-marathon.csv")
pp.ProfileReport(df, check_correlation=True)

Number of variables,21
Number of observations,31984
Total Missing (%),5.0%
Total size in memory,5.1 MiB
Average record size in memory,168.0 B
Numeric,4
Categorical,14
Boolean,0
Date,0
Text (Unique),1
Rejected,2


In [3]:
df['5k'] = np.where(df['5k'] == '-' , 0, df['5k']).astype(float)
df['10k'] = np.where(df['10k'] == '-' , 0, df['10k']).astype(float)
df['20k'] = np.where(df['20k'] == '-' , 0, df['20k']).astype(float)
df['half'] = np.where(df['half'] == '-' , 0, df['half']).astype(float)
df['25k'] = np.where(df['25k'] == '-' , 0, df['25k']).astype(float)
df['30k'] = np.where(df['30k'] == '-' , 0, df['30k']).astype(float)
df['35k'] = np.where(df['35k'] == '-' , 0, df['35k']).astype(float)
df['40k'] = np.where(df['40k'] == '-' , 0, df['40k']).astype(float)

# Cleaning: Fill NaNs
values_to_fill = {
    'state': 'Other',
    'ctz': 'NA'
    }
df = df.fillna(value=values_to_fill)

pp.ProfileReport(df, check_correlation=False)

Number of variables,21
Number of observations,31984
Total Missing (%),0.0%
Total size in memory,5.1 MiB
Average record size in memory,168.0 B
Numeric,14
Categorical,6
Boolean,0
Date,0
Text (Unique),1
Rejected,0


In [4]:
# feature eng

# Define X
X = df.drop(columns=[
                    'name', # has a high cardinality: 31915 distinct values
                    'city', # is categorical 
                    'country', # is categorical 
                    'ctz', # is categorical 
                    'state', # is categorical 
                    'bib', # is categorical 
                    'gender' # is categorical 
                    ])
X = pd.concat([X, pd.get_dummies(df['city'])], axis=1)
X = pd.concat([X, pd.get_dummies(df['country'])], axis=1)
X = pd.concat([X, pd.get_dummies(df['ctz'])], axis=1)
X = pd.concat([X, pd.get_dummies(df['state'])], axis=1)
X = pd.concat([X, pd.get_dummies(df['gender'])], axis=1)

In [5]:
from sklearn.model_selection import train_test_split
# PCA
# Normalize

#Divide into training and test sets.
X_train, X_test = train_test_split(
    X,
    test_size=0.9,
    random_state=42)

In [ ]:
X_train.info()

In [10]:
from sklearn.cluster import MeanShift, estimate_bandwidth

# Here we set the bandwidth. This function automatically derives a bandwidth
# number based on an inspection of the distances among points in the data.
bandwidth = estimate_bandwidth(X_train, quantile=0.2, n_samples=500)

# Declare and fit the model.
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(X_train)

# Extract cluster assignments for each data point.
labels = ms.labels_
print(list(labels).count())
# Coordinates of the cluster centers.
cluster_centers = ms.cluster_centers_

# Count our clusters.
n_clusters_ = len(np.unique(labels))

print("Number of estimated clusters: {}".format(n_clusters_))


TypeError: count() takes exactly one argument (0 given)

In [ ]:
print('Comparing the assigned categories to the ones in the data:')
print(pd.crosstab(y_train,labels))

In [9]:
from sklearn.cluster import SpectralClustering

# We know we're looking for three clusters.
n_clusters=2

# Declare and fit the model.
sc = SpectralClustering(n_clusters=n_clusters)
sc.fit(X_train)

#Predicted clusters.
predict=sc.fit_predict(X_train)

#Graph results.
plt.scatter(X_train[:, 0], X_train[:, 1], c=predict)
plt.show()

print('Comparing the assigned categories to the ones in the data:')
print(pd.crosstab(y_train,predict))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/manifold/spectral_embedding_.py:237: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/_methods.py:121: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:249: RuntimeWarning: invalid value encountered in add
  distances += XX
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:249: RuntimeWarning: invalid value encountered in add
  distances += XX
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:249: RuntimeWarning: invalid value encountered in add
  distances += XX
/L

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/manifold/spectral_embedding_.py:237: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/_methods.py:121: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:249: RuntimeWarning: invalid value encountered in add
  distances += XX
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:249: RuntimeWarning: invalid value encountered in add
  distances += XX
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:249: RuntimeWarning: invalid value encountered in add
  distances += XX
/L

TypeError: '(slice(None, None, None), 0)' is an invalid key

In [ ]:
from sklearn.cluster import AffinityPropagation
from sklearn import metrics

# Declare the model and fit it in one statement.
# Note that you can provide arguments to the model, but we didn't.
af = AffinityPropagation().fit(X_train)
print('Done')

# Pull the number of clusters and cluster assignments for each data point.
cluster_centers_indices = af.cluster_centers_indices_
n_clusters_ = len(cluster_centers_indices)
labels = af.labels_

print('Estimated number of clusters: {}'.format(n_clusters_))


In [ ]:
from itertools import cycle

plt.figure(1)
plt.clf()

# Cycle through each cluster and graph them with a center point for the
# exemplar and lines from the exemplar to each data point in the cluster.
colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
for k, col in zip(range(n_clusters_), colors):
    class_members = labels == k
    cluster_center = X_train[cluster_centers_indices[k]]
    plt.plot(X_train[class_members, 0], X_train[class_members, 1], col + '.')
    plt.plot(cluster_center[0],
             cluster_center[1],
             'o',
             markerfacecolor=col,
             markeredgecolor='k')
    for x in X_train[class_members]:
        plt.plot([cluster_center[0], x[0]], [cluster_center[1], x[1]], col)

plt.title('Estimated number of clusters: {}'.format(n_clusters_))
plt.show()